<a href="https://colab.research.google.com/github/jhsrojasro/Stochastic-Music-using-HMM/blob/master/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Música Estocastica usando Modelo Oculto de Markov
Camilo Andres Contreras Cristancho

Danier Elian Gonzalez Ordoñez

Jhon Sebastian Rojas Rodriguez

Juan Sebastian Nieto Giraldo

Yerson Andres Valderrama Ceron

Modelos y Simulación 2020-1

#Creación del modelo




In [1]:
!pip install --user git+https://github.com/hmmlearn/hmmlearn.git
#Se debe reiniciar el entorno de ejecucion luego de clonar el repositorio

  Cloning https://github.com/hmmlearn/hmmlearn.git to /tmp/pip-req-build-3jxtrrlv
  Running command git clone -q https://github.com/hmmlearn/hmmlearn.git /tmp/pip-req-build-3jxtrrlv
  Created wheel for hmmlearn: filename=hmmlearn-0.2.3.post12+g7733adc-cp36-cp36m-linux_x86_64.whl size=324761 sha256=bab777e373f0ca80f871156eea7628ff239775c3d0514930cb667f18b68d4215
  Stored in directory: /tmp/pip-ephem-wheel-cache-57fz0fnz/wheels/fd/a5/f8/67f3bca7b157c6a0fef609c7d6e7135c8994abeba54aee76c6
Successfully built hmmlearn


In [1]:
import numpy as np
import math
from hmmlearn import hmm
import pickle
from music21 import *

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


path de los datos

In [3]:
generos = ["jazz", "salsa", "rock", "blues"]
path = "/content/drive/Shared drives/Modelos y Simulación/Dataset"
modelos = {}
scores = {}

Cargamos los datos en diccionarios

In [4]:
datos = {}
observations = {}
n_observations = {}
n_lengths = {}
leng = {}
for genero in generos:
  datos[genero] = np.transpose([np.loadtxt((path + "/data_"+genero+"_tr.txt"),dtype=int)])
  observations[genero] = ((open(path + "/states_"+genero+".txt", "r")).readline()).split()
  n_observations[genero] = len(observations[genero])
  n_lengths[genero] = [int(x) for x in ((open(path + "/lengths_"+genero+".txt")).readline()).split()]
  leng[genero] = np.sum(n_lengths[genero])

In [5]:
for genero in generos:
  s = np.sum(n_lengths[genero])
  print(genero,": ",s," train: ",np.floor(s*0.25))

jazz :  2461  train:  615.0
salsa :  1072  train:  268.0
rock :  1043  train:  260.0
blues :  1405  train:  351.0


Búsqueda de hiperparámetros por medio de los criterios AIC y BIC.

In [6]:
def AIC(k, lg):
  return -2*lg + 2*k

def BIC(k, lg, n):
  return -2*lg + k * np.log(n)

In [ ]:
resultados = {}
best_aic = {}
best_bic = {}
for genero in generos:
  results = []

  baic = (5, math.inf)
  bbic = (6, math.inf)

  #prueba con cada numero de estados hasta el numero de observaciones
  for states in range(1, n_observations[genero] + 1):
    k = states * (1 + states + n_observations[genero])
    bestlog = -math.inf
    #se repite el entrenamiento diez veces para obtener el mejor modelo con ese numero de estados
    for i in range(10):
      model = hmm.MultinomialHMM(n_components=states)
      model.n_features = n_observations[genero]
      model.fit(datos[genero], n_lengths[genero])
      logprob = model.score(datos[genero], n_lengths[genero])
      if(logprob > bestlog):
        bestlog = logprob
    #calclo de los scores de AIC y BIC del mejor modelo con ese numero de estados    
    aic = AIC(k, bestlog)
    bic = BIC(k, bestlog, leng[genero])
    results.append( (states, aic, bic) )
    if(aic < baic[1]):
      baic = (states, aic)
    if(bic < bbic[1]):
      bbic = (states, bic)
  resultados[genero] = results
  best_aic[genero] = baic
  best_bic[genero] = bbic

Imprimimos el resultado de la búsqueda de Hiperparámetros.

In [ ]:
for genero in generos:
  #print("Genero: ",genero)
  #for resultado in resultados[genero]:
  #  print(resultado[2])
 print("Menor AIC ",genero,": ",best_aic[genero][1]," con ",best_aic[genero][0]," estados")
 print("Menor BIC ",genero,": ",best_bic[genero][1]," con ",best_bic[genero][0]," estados")

Menor AIC:  8603.240058476626  con  10  estados
Menor BIC:  10388.970298948938  con  5  estados
Menor AIC:  4212.346843597803  con  10  estados
Menor BIC:  5064.992245322619  con  4  estados
Menor AIC:  4370.950563897886  con  9  estados
Menor BIC:  5109.73968753433  con  3  estados
Menor AIC:  4156.7703163826645  con  11  estados
Menor BIC:  5435.468448124743  con  4  estados


Entrenamiento y selección de los mejores modelos con los hiperparametros obtenidos por cada criterio.

In [ ]:
best_states = {"jazz" : 10, "salsa" : 10, "rock" : 9, "blues" : 11}
n_entrenamientos = 25
for genero in generos:
  X_train = np.transpose([np.loadtxt((path + "/train_"+genero+".txt"),dtype=int)])
  X_test = np.transpose([np.loadtxt((path + "/test_"+genero+".txt"),dtype=int)])
  train_len = [int(x) for x in ((open(path + "/train_"+genero+"_len.txt")).readline()).split()]
  test_len = [int(x) for x in ((open(path + "/test_"+genero+"_len.txt")).readline()).split()]

  best_score = -math.inf
  best_model = hmm.MultinomialHMM(n_components=n_observations[genero], n_iter=1000)

  for j in range(n_entrenamientos):
    model = hmm.MultinomialHMM(n_components=best_states[genero], n_iter=1000)
    model.n_features = n_observations[genero]
    model.fit(X_train, train_len)
    score = model.score(X_train, train_len)
    if(score > best_score):
      best_model = model
      best_score = score
  print("best score ",genero,": ",best_score)
  scores[genero] = (best_score , best_model.score(X_test, test_len))
  modelos[genero] = best_model


best score:  -1112.3412119453405


Resultados del entrenamiento

In [ ]:
for genero in generos:
  print("Genero: ",genero)
  print("best train log likehood: ",scores[genero][0]," test log likelihood: ",scores[genero][1] )

Genero:  jazz
best train log likehood:  -2507.1172601719077  test log likelihood:  -880.9250271151368
Genero:  salsa
best train log likehood:  -1025.8035924706644  test log likelihood:  -517.3501756066039
Genero:  rock
best train log likehood:  -1177.964165091988  test log likelihood:  -491.181476785466
Genero:  blues
best train log likehood:  -1112.3412119453405  test log likelihood:  -270.03358418142363


Codigo para serializar el mejor modelo obtenido para cada genero.

In [ ]:
for genero in generos:
  with open("/content/drive/Shared drives/Modelos y Simulación/BestModels/"+genero+"_model.pkl", "wb") as file: pickle.dump(modelos[genero], file)

Codigo para cargar los mejores modelos.

In [ ]:
for genero in generos:
  modelos[genero] = pickle.load(open("/content/drive/Shared drives/Modelos y Simulación/BestModels/"+genero+"_model.pkl", "rb"))

Celda para generar una secuencia de salida a partir de un modelo entrenado

In [ ]:
genero = "jazz"
longitud_ejemplo = 79*2

Y, ejemplos = modelos[genero].sample(longitud_ejemplo)
ej = (" ".join(map(lambda x: observations[genero][x], ejemplos))).split()
print(ej)
#print("Ejemplos", ", ".join(map(lambda x: observations[x], ejemplos)))

#Generación de una salida audible.

Funciones que generan un ritmo aleatorio para cada nota de cada acorde producido por el modelo.

In [ ]:
durations = [duration.Duration(1), duration.Duration(1/2),
             duration.Duration(1/3), duration.Duration(3/4),
             duration.Duration(1/4), duration.Duration(2/3)]
def negra(p, s):
  s.append(note.Note(pitch=p, duration=durations[0]))

def corcheas(p, s):
  n = note.Note(pitch=p, duration=durations[1])
  s.repeatAppend(n, 2)


def tresillos1(p, s):
  n1 = note.Note(pitch=p, duration=durations[2])
  n2 = note.Note(pitch=p, duration=durations[5])
  s.append(n1)
  s.append(n2)


def tresillos2(p, s):
  n1 = note.Note(pitch=p, duration=durations[5])
  n2 = note.Note(pitch=p, duration=durations[2])
  s.append(n1)
  s.append(n2)

def corcheap1(p, s):
  n1 = note.Note(pitch=p, duration=durations[3])
  n2 = note.Note(pitch=p, duration=durations[4])
  s.append(n1)
  s.append(n2)

def corcheap2(p, s):
  n1 = note.Note(pitch=p, duration=durations[4])
  n2 = note.Note(pitch=p, duration=durations[3])
  s.append(n1)
  s.append(n2)

def ritmo(p, s):
  asign = {0: negra, 1: corcheas, 2: tresillos1, 3: corcheap1, 4: corcheap2, 5: tresillos2}
  for i in range(2):
    r = np.random.randint(6)
    asign[r](p, s)
  

Inicialización de los instrumentos y asignación de un conjunto de instrumentos a cada género.

In [ ]:
piano = instrument.Piano()
contrabajo = instrument.AcousticBass()
bajo = instrument.ElectricBass()
trompeta = instrument.Trumpet()
saxo = instrument.Saxophone()
clarinete = instrument.Clarinet()
trombon = instrument.Trombone()
guitarra = instrument.ElectricGuitar()
aguitarra = instrument.AcousticGuitar()
voz = instrument.Vocalist()
violin = instrument.Violin()


instrumentos = {
    "jazz" : [piano, contrabajo, saxo, trompeta, clarinete],
    "salsa" : [piano, contrabajo, trombon, saxo, trompeta],
    "rock" : [piano, bajo, guitarra, guitarra, voz],
    "blues" : [piano, contrabajo, aguitarra, saxo, violin ]
}


Insersión de cada nota en cada parte de la partitura de acuerdo al ejemplo generado por el modelo seleccionado. Y agrupación de las partes en un solo Stream Midi

In [ ]:
partes = [stream.Part(), stream.Part(), stream.Part(), stream.Part(), stream.Part()]
for i in range(len(partes)):
  partes[i].insert(0, instrumentos[genero][i])

for chord in ej:
  c = roman.RomanNumeral(chord)
  c.duration = duration.Duration(2)
  partes[0].append(c)
  c.pitches[0].octave -= 2
  ritmo(c.pitches[0], partes[1])
  ritmo(c.pitches[1], partes[2])
  ritmo(c.pitches[ 1 if len(c.pitches) == 3 else 2 ], partes[3])
  ritmo(c.pitches[ 2 if len(c.pitches) == 3 else 3 ], partes[4])

midi_stream = stream.Stream()
for parte in partes:
  midi_stream.insert(0, parte)


Escritura del archivo MIDI de acuerdo al Stream anterior

In [ ]:
#midi_stream.show()
midi_stream.write('midi', fp="/content/drive/Shared drives/Modelos y Simulación/Ejemplos/"+genero+"_output.mid")

'/content/drive/Shared drives/Modelos y Simulación/Ejemplos/blues_output.mid'